<h1> AutoAI4EO: NAS with AutoKeras for Earth Observation (Part 2) <h1>

<p>This is the second blog post in the series about our research on Neural Architecture Search (NAS) for Earth Observation (EO). In Part 1 < link > we introduced NAS and how it can be applied to EO. We talked about a NAS framework: AutoKeras [1], and briefly discussed how it can be customized for EO tasks. In this blog post, we are going to dive more in-depth into how AutoKeras can be used to create methods for EO imagery. More specifically, we are going to tackle the task of classification of EO imagery through the work “Automated Machine Learning for Satellite Data: Integrating Remote Sensing Pre-trained Models into AutoML Systems” by Nelly R. Palacios Salinas, Mitra Baratchi, Jan N. van Rijn and Andreas Vollrath [2].</p>

<h2> Classification </h2>
<p> Image classification is the task of assigning labels to an image. For instance, you’d like to know whether your image is showing a desert or a forest. Classification has, among others, applications in tasks like urban planning, hazard detection, and monitoring of the environment [3]. Sometimes in EO the term “image classification” is used to refer to what is called segmentation in computer science. This is the task of assigning labels to individual pixels. For instance, you want to know of each individual pixel whether it is part of a building or not. However, we will only talk about classification in the traditional sense, where you consider the complete image. </p>

In [3]:
import autokeras as ak
import tensorflow as tf

In [4]:
class HyperBlock(ak.Block):
        
    def build(self, hp, inputs):
        inputs = tf.nest.flatten(inputs)[0]
        
        if hp.Choice("model_type",["resnet", "xception"]) == "resnet":
            outputs = ak.ResNetBlock().build(hp,inputs)
        else:
            outputs=ak.XceptionBlock().build(hp,inputs)
        return outputs

In [8]:
from typing import Optional

class HyperBlock(ak.Block):
    def __init__(self, model_type: Optional[str] = None):
        super().__init__(**kwargs)
        
        if model_type is not None and model_type != "resnet" and model_type != "wdsr":
            raise Exception(f"invalid model_type {model_type}")

        self.model_type=model_type

    def get_config(self):
        config = super().get_config()
        config.update({"model_type": self.model_type})
        return config

    def _build_model(self,hp, output_node,model_type: str):
        if model_type=="resnet":
            return ak.ResNetBlock().build(hp,output_node)
        elif model_type=="xception":
            return ak.XceptionBlock().build(hp, output_node)

    def build(self, hp, inputs):
        inputs=tf.nest.flatten(inputs)[0]

        # Let AutoKeras choose a model
        if self.model_type is None:
            model_type= hp.Choice("model_type", ["resnet", "xception"])
            with hp.conditional_scope("model_type",[model_type]):
                outputs = self._build_model(hp,inputs,model_type)
        # Select model yourself
        else:
            outputs = self._build_model(hp,inputs,model_type)

In [12]:

EO_VERSIONS= {
    "resisc45": "https://tfhub.dev/google/remote_sensing/resisc45-resnet50/1",
    "eurosat": "https://tfhub.dev/google/remote_sensing/eurosat-resnet50/1",
    "so2sat": "https://tfhub.dev/google/remote_sensing/so2sat-resnet50/1",
    "ucmerced": "https://tfhub.dev/google/remote_sensing/uc_merced-resnet50/1",
}


In [25]:
set(["eurosat", "so2sat"]) <= set(EO_VERSIONS.keys())

True

In [26]:
import tensorflow_hub as hub

class EOResNetBlock(ak.Block):
    #Remote sensing pretrained modules based on: https://github.com/palaciosnrps/automl-rs-project/blob/714bbe36c68fd0f2b989bfee89eac9497d7acf45/autokeras/blocks/basic.py"""
    def __init__(
         self, 
         version: Optional[str] = None,
         **kwargs,
         ):
            super().__init__(**kwargs)

            if version is not None and version not in EO_VERSIONS.keys() and set(version) <= set(EO_VERSIONS.keys()):
                raise Exception(f"invalid version {version}")
        
            self.version=version

    def get_config(self):
        config = super().get_config()
        config.update({"version": self.version})
        return config

    def build(self, hp, inputs=None):
        # Get the input_node from inputs.

        input_node = tf.nest.flatten(inputs)[0]

        if self.version is None:
            version= hp.Choice("version", list(EO_VERSIONS.keys()))
        elif isinstance(self.version,list):
            version = self.version
        else:
            version = [self.version]
            
        module = hub.KerasLayer(EO_VERSIONS[version],tags='train',trainable=False)
        min_size = 224
        if input_node.shape[3] not in [1, 3]:
            if self.pretrained:
                raise ValueError(
                    "When pretrained is set to True, expect input to "
                    "have 1 or 3 channels, bug got "
                    "{channels}.".format(channels=input_node.shape[3])
                )

        if input_node.shape[1] < min_size or input_node.shape[2] < min_size:
            input_node = tf.keras.layers.experimental.preprocessing.Resizing(
                max(min_size, input_node.shape[1]),
                max(min_size, input_node.shape[2]),
            )(input_node)
        if input_node.shape[3] == 1:
            input_node = tf.keras.layers.Concatenate()([input_node] * 3)
        if input_node.shape[3] != 3:
            input_node = tf.keras.layers.Conv2D(filters=3, kernel_size=1, padding="same")(
                input_node
            )	

        output_node = module(input_node)
        return output_node

In [27]:
class HyperBlock(ak.Block):
    def __init__(self, model_type: Optional[str] = None):
        super().__init__(**kwargs)
        
        if model_type is not None and model_type != "resnet" and model_type != "wdsr" and model_type != "eo_resnet":
            raise Exception(f"invalid model_type {model_type}")

        self.model_type=model_type

    def get_config(self):
        config = super().get_config()
        config.update({"model_type": self.model_type})
        return config

    def _build_model(self,hp, output_node,model_type: str):
        if model_type=="resnet":
            return ak.ResNetBlock().build(hp,output_node)
        elif model_type=="xception":
            return ak.XceptionBlock().build(hp, output_node)
        elif model_type=="eo_resnet":
            return EOResNetBlock().build(hp,output_node)

    def build(self, hp, inputs):
        inputs=tf.nest.flatten(inputs)[0]

        # Let AutoKeras choose a model
        if self.model_type is None:
            model_type= hp.Choice("model_type", ["resnet", "xception", "eo_resnet"])
            with hp.conditional_scope("model_type",[model_type]):
                outputs = self._build_model(hp,inputs,model_type)
        # Select model yourself
        else:
            outputs = self._build_model(hp,inputs,model_type)

In [28]:
{k: v for k,v in EO_VERSIONS.items() if k!= "ucmerced"}

{'resisc45': 'https://tfhub.dev/google/remote_sensing/resisc45-resnet50/1',
 'eurosat': 'https://tfhub.dev/google/remote_sensing/eurosat-resnet50/1',
 'so2sat': 'https://tfhub.dev/google/remote_sensing/so2sat-resnet50/1'}

In [30]:
import tensorflow_datasets as tfds
# load the dataset 317.MiB
train_set=tfds.load("uc_merced", as_supervised=False, batch_size=-1,split="all[:80%]")
test_set=tfds.load("uc_merced", as_supervised=False, batch_size=-1,split="all[80%:]")

weights_versions = {k: v for k,v in EO_VERSIONS.items() if k != "ucmerced"}

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
input_node = ak.ImageInput()
output_node = HyperBlock(model_type="eo_resnet")(input_node)
output_node = ak.ClassificationHead()(output_node)
eo_nas_model = ak.AutoModel(input_node, output_node, max_trials=1,overwrite=True)

eo_nas_model.fit(x=train_set["image"], y=train_set["label"])